In [7]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import r2_score


import matplotlib.pyplot as plt
import seaborn as sns

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from sklearn import linear_model
from sklearn import svm
from sklearn.svm import SVC

In [8]:
file = pd.read_csv('../framinghan_function/framingham-score-risk.csv')
df   = pd.DataFrame(file)

In [9]:
df = df.drop(columns = ['ap_hi', 'ap_lo', 'cholesterol', 'cardio', 'fram_score', 'risk'])

In [10]:
df

,age,gender,BMI,gluc,smoke,alco,active,risk_group
0,50.0,2,22.0,1,0,0,1,2
1,55.0,1,35.0,1,0,0,1,4
2,52.0,1,24.0,1,0,0,0,3
3,48.0,2,29.0,1,0,0,1,3
4,48.0,1,23.0,1,0,0,0,1
...,...,...,...,...,...,...,...,...
69995,53.0,2,27.0,1,1,0,1,3
69996,62.0,1,50.0,2,0,0,1,5
69997,52.0,2,31.0,1,0,1,0,5
69998,61.0,1,27.0,2,0,0,0,4


In [5]:
X = df.drop(["risk_group"], axis=1)
y = df["risk_group"]

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=202)

In [13]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

# Tuning hyper-parameters for precision



c:\users\miguestepa\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\miguestepa\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\miguestepa\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.377 (+/-0.155) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.063 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.559 (+/-0.013) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.377 (+/-0.155) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.565 (+/-0.012) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.558 (+/-0.013) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.582 (+/-0.007) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.564 (+/-0.013) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.571 (+/-0.009) for {'C': 1, 'kernel': 'linear'}
0.579 (+/-0.008) for {'C': 10, 'kernel': 'linear'}
0.579 (+/-0.007) for {'C': 100, 'kernel': 'linear'}
0.580 (+/-0.008) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

           

In [15]:
f = open("SVC-5p-best-score", "a")
print("# Tuning hyper-parameters for %s" % score, file=f)
print("Best parameters set found on development set:", file=f)
print('', file=f)
print(clf.best_params_ , file=f)
print('', file=f)
print("Grid scores on development set:", file=f)
print('', file=f)
print("Detailed classification report:", file=f)
print('', file=f)
print(classification_report(y_true, y_pred), file=f)
f.close()


In [11]:
#Create a svm Classifier
clf = svm.SVC(C = 1000, gamma=0.001, kernel='rbf') # results from the hiperparameter tunning

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [12]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5620714285714286
